
Sistema de Recomendação de Produtos Agrícolas do DF
Projeto 1 - Introdução à Inteligência Artificial
Professor Díbio - UnB 2025/1

Raul Myron Silva Amorim - 200049712
Lucca Santos Aguilar - 241024221
Thais Aragão Bianchini - 

Este notebook implementa um sistema de recomendação para
pequenos produtores rurais a consumidores no Distrito Federal.


In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import folium
import json
from datetime import datetime
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
"""
Dados extraídos manualmente dos documentos fornecidos:
- 17 Associações/Cooperativas com coordenadas geográficas
- Produtos oferecidos por cada uma (baseado nos editais do GDF)
- Regiões de atuação no DF
"""

# Lista completa de produtos do escopo do projeto
produtos_escopo = [
    'Alface', 'Mandioca', 'Tomate', 'Repolho', 'Batata', 'Cebola', 'Couve',
    'Chuchu', 'Morango', 'Pimentão', 'Brócolis', 'Abóbora', 'Berinjela',
    'Beterraba', 'Pepino', 'Cenoura', 'Quiabo', 'Agrião', 'Jiló', 'Gengibre',
    'Abacate', 'Goiaba', 'Banana', 'Limão', 'Tangerina', 'Maracujá', 'Manga',
    'Lichia', 'Uva', 'Atemóia', 'Cajamanga', 'Graviola', 'Coco', 'Pitaia', 'Mamão'
]

# Mapeamento entre nomes dos produtos nos documentos e produtos do escopo
mapeamento_produtos = {
    'Abóbora Japonesa': 'Abóbora',
    'Abobrinha Italiana': 'Abóbora',
    'Acelga': 'Couve',
    'Alface Americana': 'Alface',
    'Cebolinha Comum': 'Cebola',
    'Coentro': 'Couve',
    'Couve-Flor': 'Brócolis',
    'Couve Manteiga': 'Couve',
    'Espinafre': 'Couve',
    'Hortelã': 'Agrião',
    'Manjericão': 'Agrião',
    'Pepino Comum': 'Pepino',
    'Pimentão Verde': 'Pimentão',
    'Repolho Verde': 'Repolho',
    'Repolho Roxo': 'Repolho',
    'Salsa': 'Agrião',
    'Batata Doce': 'Batata',
    'Brócolis Cabeça Única (Japonês)': 'Brócolis',
    'Inhame': 'Batata',
    'Limão Tahiti': 'Limão',
    'Milho Verde': 'Pepino',
    'Tangerina Ponkan': 'Tangerina',
    'Vagem': 'Pepino',
    'Banana Prata': 'Banana',
    'Alho': 'Cebola'
}

# Dados das 17 associações/cooperativas
associacoes_data = [
    {
        'id': 1,
        'nome': 'AFECA - Assentamento 15 de Agosto',
        'latitude': -15.911319,
        'longitude': -47.721311,
        'regioes': ['São Sebastião'],
        'produtos_originais': ['Abóbora Japonesa', 'Abobrinha Italiana', 'Acelga', 'Alface Americana', 
                              'Cebolinha Comum', 'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 
                              'Espinafre', 'Hortelã', 'Manjericão', 'Maracujá', 'Pepino Comum', 
                              'Pimentão Verde', 'Repolho Verde', 'Repolho Roxo', 'Salsa', 'Tomate']
    },
    {
        'id': 2,
        'nome': 'AGRIFAM - Agricultura Familiar DF',
        'latitude': -15.932992,
        'longitude': -48.040909,
        'regioes': ['Taguatinga', 'Gama', 'Santa Maria'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Acelga', 
                              'Alface Americana', 'Alho', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebola', 'Cebolinha Comum', 
                              'Cenoura', 'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 
                              'Espinafre', 'Goiaba', 'Hortelã', 'Inhame', 'Limão Tahiti', 
                              'Manjericão', 'Maracujá', 'Milho Verde', 'Pepino Comum', 
                              'Pimentão Verde', 'Repolho Verde', 'Repolho Roxo', 'Salsa', 
                              'Tangerina Ponkan', 'Tomate', 'Vagem', 'Banana Prata']
    },
    {
        'id': 3,
        'nome': 'AMISTA - Agricultores Orgânicos',
        'latitude': -15.782057,
        'longitude': -47.470923,
        'regioes': ['Santa Maria'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Batata Doce', 
                              'Beterraba', 'Brócolis Cabeça Única (Japonês)', 'Cebolinha Comum', 
                              'Cenoura', 'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 
                              'Inhame', 'Limão Tahiti', 'Maracujá', 'Pepino Comum', 'Pimentão Verde', 
                              'Repolho Verde', 'Repolho Roxo', 'Salsa', 'Tomate', 'Vagem']
    },
    {
        'id': 4,
        'nome': 'ASPAF - Produtores Agricultura Familiar',
        'latitude': -15.940404,
        'longitude': -47.737964,
        'regioes': ['Guará', 'Núcleo Bandeirante', 'Plano Piloto'],
        'produtos_originais': ['Goiaba', 'Morango', 'Tangerina Ponkan', 'Abacate', 'Abóbora Japonesa', 
                              'Abobrinha Italiana', 'Acelga', 'Alface Americana', 'Batata Doce', 
                              'Beterraba', 'Brócolis Cabeça Única (Japonês)', 'Cebola', 
                              'Cebolinha Comum', 'Cenoura', 'Chuchu', 'Coentro', 'Couve-Flor', 
                              'Couve Manteiga', 'Espinafre', 'Hortelã', 'Inhame', 'Limão Tahiti', 
                              'Manjericão', 'Maracujá', 'Milho Verde', 'Pepino Comum', 
                              'Pimentão Verde', 'Repolho Verde', 'Repolho Roxo', 'Salsa', 
                              'Tomate', 'Vagem', 'Banana Prata']
    },
    {
        'id': 5,
        'nome': 'ASPAG - Alexandre Gusmão',
        'latitude': -15.726869,
        'longitude': -48.189019,
        'regioes': ['Alexandre Gusmão', 'Brazlândia'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Acelga', 
                              'Alface Americana', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebolinha Comum', 'Cenoura', 
                              'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 'Espinafre', 
                              'Goiaba', 'Hortelã', 'Inhame', 'Limão Tahiti', 'Manjericão', 
                              'Maracujá', 'Milho Verde', 'Pepino Comum', 'Pimentão Verde', 
                              'Repolho Verde', 'Repolho Roxo', 'Salsa', 'Tangerina Ponkan', 
                              'Tomate', 'Vagem', 'Banana Prata']
    },
    {
        'id': 6,
        'nome': 'ASPHOR - Hortigranjeiros DF',
        'latitude': -15.791442,
        'longitude': -47.948464,
        'regioes': ['Plano Piloto', 'Gama', 'Santa Maria'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Acelga', 
                              'Alface Americana', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebola', 'Cebolinha Comum', 
                              'Cenoura', 'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 
                              'Espinafre', 'Goiaba', 'Hortelã', 'Inhame', 'Limão Tahiti', 
                              'Manjericão', 'Maracujá', 'Milho Verde', 'Morango', 'Pepino Comum', 
                              'Pimentão Verde', 'Repolho Verde', 'Repolho Roxo', 'Salsa', 
                              'Tomate', 'Vagem']
    },
    {
        'id': 7,
        'nome': 'ASPROC - Orgânicos e Convencionais',
        'latitude': -15.777904,
        'longitude': -48.120988,
        'regioes': ['Ceilândia', 'Recanto das Emas', 'Samambaia', 'Paranoá', 'Planaltina'],
        'produtos_originais': ['Goiaba', 'Tangerina Ponkan', 'Morango']
    },
    {
        'id': 8,
        'nome': 'ASPRONTE - Novo Horizonte',
        'latitude': -15.603130,
        'longitude': -48.113516,
        'regioes': ['Brazlândia', 'Ceilândia', 'Recanto Das Emas'],
        'produtos_originais': ['Morango', 'Abacate', 'Abobrinha Italiana', 'Acelga', 
                              'Alface Americana', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebolinha Comum', 'Cenoura', 
                              'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 'Espinafre', 
                              'Goiaba', 'Hortelã', 'Pepino Comum', 'Pimentão Verde', 
                              'Repolho Verde', 'Salsa', 'Tomate', 'Vagem']
    },
    {
        'id': 9,
        'nome': 'ASTRAF - Assentamento Chapadinha',
        'latitude': -15.542238,
        'longitude': -48.029995,
        'regioes': ['PAD-DF', 'Planaltina'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Acelga', 
                              'Alface Americana', 'Alho', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebola', 'Cebolinha Comum', 
                              'Cenoura', 'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 
                              'Espinafre', 'Hortelã', 'Inhame', 'Limão Tahiti', 'Manjericão', 
                              'Maracujá', 'Milho Verde', 'Pepino Comum', 'Pimentão Verde', 
                              'Repolho Verde', 'Repolho Roxo', 'Salsa', 'Tomate', 'Vagem', 
                              'Banana Prata']
    },
    {
        'id': 10,
        'nome': 'COOPBRASIL - Agricultura Familiar',
        'latitude': -15.918585,
        'longitude': -48.082794,
        'regioes': ['Recanto das Emas', 'Gama', 'Samambaia', 'Núcleo Bandeirante', 
                   'Planaltina', 'Brazlândia', 'Ceilândia'],
        'produtos_originais': ['Alho', 'Abóbora Japonesa', 'Cebola', 'Inhame', 'Limão Tahiti', 
                              'Manjericão', 'Maracujá', 'Milho Verde', 'Repolho Roxo', 
                              'Banana Prata', 'Abacate', 'Abobrinha Italiana', 'Acelga', 
                              'Alface Americana', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebolinha Comum', 'Cenoura', 
                              'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 'Espinafre', 
                              'Goiaba', 'Hortelã', 'Pepino Comum', 'Pimentão Verde', 
                              'Repolho Verde', 'Salsa', 'Tomate', 'Vagem', 'Morango']
    },
    {
        'id': 11,
        'nome': 'Cooper-Horti - Buriti Vermelho',
        'latitude': -15.898749,
        'longitude': -47.409203,
        'regioes': ['Paranoá', 'PAD-DF'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Batata Doce', 
                              'Beterraba', 'Cenoura', 'Chuchu', 'Couve Manteiga', 'Inhame', 
                              'Limão Tahiti', 'Maracujá', 'Pepino Comum', 'Pimentão Verde', 
                              'Repolho Verde', 'Tangerina Ponkan']
    },
    {
        'id': 12,
        'nome': 'Prorural - Planaltina GO',
        'latitude': -15.445505,
        'longitude': -47.619013,
        'regioes': ['Planaltina', 'Plano Piloto', 'Paranoá'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Acelga', 
                              'Alface Americana', 'Alho', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebola', 'Cebolinha Comum', 
                              'Cenoura', 'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 
                              'Espinafre', 'Goiaba', 'Hortelã', 'Inhame', 'Limão Tahiti', 
                              'Manjericão', 'Maracujá', 'Milho Verde', 'Pepino Comum', 
                              'Pimentão Verde', 'Repolho Verde', 'Repolho Roxo', 'Salsa', 
                              'Tomate', 'Vagem', 'Banana Prata']
    },
    {
        'id': 13,
        'nome': 'Coopebraz - Brazlândia',
        'latitude': -15.602622,
        'longitude': -48.113771,
        'regioes': ['Brazlândia', 'Taguatinga', 'Samambaia', 'Recanto Das Emas'],
        'produtos_originais': ['Morango']
    },
    {
        'id': 14,
        'nome': 'Coopermista - Agricultura Familiar',
        'latitude': -15.764000,
        'longitude': -47.493246,
        'regioes': ['Planaltina'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Batata Doce', 
                              'Beterraba', 'Cenoura', 'Couve Manteiga', 'Limão Tahiti', 
                              'Maracujá', 'Pepino Comum', 'Repolho Verde', 'Repolho Roxo', 
                              'Tangerina Ponkan']
    },
    {
        'id': 15,
        'nome': 'Rede Terra - Ecológicos do Cerrado',
        'latitude': -16.784921,
        'longitude': -47.600542,
        'regioes': ['Santa Maria', 'Gama'],
        'produtos_originais': ['Acelga', 'Alface Americana', 'Alho', 'Cebola', 'Espinafre', 
                              'Goiaba', 'Hortelã', 'Manjericão', 'Milho Verde']
    },
    {
        'id': 16,
        'nome': 'Cootaquara - Planaltina',
        'latitude': -15.632824,
        'longitude': -47.522650,
        'regioes': ['Planaltina', 'Ceilândia'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 
                              'Alface Americana', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebolinha Comum', 'Cenoura', 
                              'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 'Espinafre', 
                              'Inhame', 'Limão Tahiti', 'Maracujá', 'Milho Verde', 'Pepino Comum', 
                              'Pimentão Verde', 'Repolho Verde', 'Repolho Roxo', 'Salsa', 
                              'Tomate', 'Vagem', 'Banana Prata']
    },
    {
        'id': 17,
        'nome': 'Cooperbrasília - Serviços Ambientais',
        'latitude': -15.918543,
        'longitude': -48.082816,
        'regioes': ['Sobradinho', 'São Sebastião'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Acelga', 
                              'Alface Americana', 'Alho', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebola', 'Cebolinha Comum', 
                              'Cenoura', 'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 
                              'Espinafre', 'Goiaba', 'Hortelã', 'Inhame', 'Limão Tahiti', 
                              'Manjericão', 'Maracujá', 'Milho Verde', 'Morango', 'Pepino Comum', 
                              'Pimentão Verde', 'Repolho Verde', 'Repolho Roxo', 'Salsa', 
                              'Tangerina Ponkan', 'Tomate', 'Vagem', 'Banana Prata']
    }
]

In [ ]:
# Converter para DataFrame e mapear produtos
df_associacoes = pd.DataFrame(associacoes_data)

# Função para mapear produtos originais para produtos do escopo
def mapear_produtos(produtos_lista):
    produtos_mapeados = []
    for produto in produtos_lista:
        if produto in mapeamento_produtos:
            produto_mapeado = mapeamento_produtos[produto]
            if produto_mapeado in produtos_escopo and produto_mapeado not in produtos_mapeados:
                produtos_mapeados.append(produto_mapeado)
        elif produto in produtos_escopo and produto not in produtos_mapeados:
            produtos_mapeados.append(produto)
    return produtos_mapeados

# Aplicar mapeamento
df_associacoes['produtos'] = df_associacoes['produtos_originais'].apply(mapear_produtos)

# Adicionar características adicionais
np.random.seed(101)
df_associacoes['organico_principal'] = np.random.choice([True, False], size=len(df_associacoes), p=[0.3, 0.7])
df_associacoes['avaliacao_media'] = np.random.uniform(3.5, 5.0, size=len(df_associacoes))
df_associacoes['num_avaliacoes'] = np.random.randint(20, 200, size=len(df_associacoes))
df_associacoes['preco_medio_relativo'] = np.random.choice(['Baixo', 'Médio', 'Alto'], 
                                                          size=len(df_associacoes), 
                                                          p=[0.3, 0.5, 0.2])

# Ajustar algumas associações específicas como orgânicas
df_associacoes.loc[df_associacoes['nome'].str.contains('Orgânicos|Ecológicos'), 'organico_principal'] = True

print(f"✅ Dados de {len(df_associacoes)} associações/cooperativas carregados com sucesso!")
print(f"\n📊 Total de produtos únicos no escopo: {len(produtos_escopo)}")
print(f"📊 Total de produtos após mapeamento: {df_associacoes['produtos'].apply(len).sum()}")
print("\n🏢 Primeiras 3 associações:")
print(df_associacoes[['id', 'nome', 'regioes', 'produtos']].head(3))

In [ ]:
dados_taco = {
    'Abacate': {'id': 163, 'umidade': 83.8, 'energia_kcal': 96, 'proteina_g': 1.2, 
                'lipidios_g': 8.4, 'carboidratos_g': 6.0, 'fibra_g': 6.3, 'calcio_mg': 8, 
                'vitamina_c_mg': 15},
    'Abóbora': {'id': 71, 'umidade': 93.9, 'energia_kcal': 19, 'proteina_g': 1.1, 
                'lipidios_g': 0.1, 'carboidratos_g': 4.3, 'fibra_g': 1.4, 'calcio_mg': 15, 
                'vitamina_c_mg': 20},
    'Agrião': {'id': 74, 'umidade': 93.2, 'energia_kcal': 21, 'proteina_g': 1.4, 
                'lipidios_g': 0.1, 'carboidratos_g': 4.6, 'fibra_g': 1.1, 'calcio_mg': 43, 
                'vitamina_c_mg': 10},
    'Alface': {'id': 77, 'umidade': 97.2, 'energia_kcal': 9, 'proteina_g': 0.6, 
               'lipidios_g': 0.1, 'carboidratos_g': 1.7, 'fibra_g': 1.0, 'calcio_mg': 14, 
               'vitamina_c_mg': 6},
    'Batata': {'id': 89, 'umidade': 69.5, 'energia_kcal': 118, 'proteina_g': 1.3, 
               'lipidios_g': 0.1, 'carboidratos_g': 28.2, 'fibra_g': 2.6, 'calcio_mg': 21, 
               'vitamina_c_mg': 17},
    'Beterraba': {'id': 98, 'umidade': 86.0, 'energia_kcal': 49, 'proteina_g': 1.9, 
                  'lipidios_g': 0.1, 'carboidratos_g': 11.1, 'fibra_g': 3.4, 'calcio_mg': 18, 
                  'vitamina_c_mg': 24},
    'Brócolis': {'id': 101, 'umidade': 91.2, 'energia_kcal': 25, 'proteina_g': 3.6, 
                 'lipidios_g': 0.3, 'carboidratos_g': 4.0, 'fibra_g': 2.9, 'calcio_mg': 86, 
                 'vitamina_c_mg': 30},
    'Cebola': {'id': 107, 'umidade': 88.9, 'energia_kcal': 39, 'proteina_g': 1.7, 
               'lipidios_g': 0.1, 'carboidratos_g': 8.9, 'fibra_g': 2.2, 'calcio_mg': 14, 
               'vitamina_c_mg': 12},
    'Cenoura': {'id': 110, 'umidade': 90.1, 'energia_kcal': 34, 'proteina_g': 1.3, 
                'lipidios_g': 0.2, 'carboidratos_g': 7.7, 'fibra_g': 3.2, 'calcio_mg': 23, 
                'vitamina_c_mg': 11},
    'Chuchu': {'id': 113, 'umidade': 94.8, 'energia_kcal': 17, 'proteina_g': 0.7, 
               'lipidios_g': 0.1, 'carboidratos_g': 4.1, 'fibra_g': 1.3, 'calcio_mg': 12, 
               'vitamina_c_mg': 7},
    'Couve': {'id': 115, 'umidade': 90.9, 'energia_kcal': 27, 'proteina_g': 2.9, 
              'lipidios_g': 0.5, 'carboidratos_g': 4.3, 'fibra_g': 3.1, 'calcio_mg': 131, 
              'vitamina_c_mg': 35},
    'Goiaba': {'id': 197, 'umidade': 85.7, 'energia_kcal': 52, 'proteina_g': 0.9, 
              'lipidios_g': 0.5, 'carboidratos_g': 12.4, 'fibra_g': 6.3, 'calcio_mg': 5, 
              'vitamina_c_mg': 7},
   'Limão': {'id': 9004, 'umidade': 91.0, 'energia_kcal': 29, 'proteina_g': 1.1, 
             'lipidios_g': 0.3, 'carboidratos_g': 9.3, 'fibra_g': 2.8, 'calcio_mg': 26, 
             'vitamina_c_mg': 53},  # Limão Tahiti
   'Manga': {'id': 133, 'umidade': 83.5, 'energia_kcal': 65, 'proteina_g': 0.5, 
             'lipidios_g': 0.3, 'carboidratos_g': 15.0, 'fibra_g': 1.6, 'calcio_mg': 12, 
             'vitamina_c_mg': 43},
   'Maracujá': {'id': 232, 'umidade': 82.9, 'energia_kcal': 68, 'proteina_g': 2.0, 
                'lipidios_g': 2.1, 'carboidratos_g': 12.3, 'fibra_g': 1.1, 'calcio_mg': 5, 
                'vitamina_c_mg': 28},
   'Morango': {'id': 239, 'umidade': 91.5, 'energia_kcal': 30, 'proteina_g': 0.9, 
               'lipidios_g': 0.3, 'carboidratos_g': 6.8, 'fibra_g': 1.7, 'calcio_mg': 11, 
               'vitamina_c_mg': 10},
   'Pepino': {'id': 142, 'umidade': 96.8, 'energia_kcal': 10, 'proteina_g': 0.9, 
              'lipidios_g': 0.0, 'carboidratos_g': 2.0, 'fibra_g': 1.1, 'calcio_mg': 10, 
              'vitamina_c_mg': 9},
   'Pimentão': {'id': 144, 'umidade': 93.5, 'energia_kcal': 21, 'proteina_g': 1.1, 
                'lipidios_g': 0.2, 'carboidratos_g': 4.9, 'fibra_g': 2.6, 'calcio_mg': 9, 
                'vitamina_c_mg': 8},
   'Repolho': {'id': 150, 'umidade': 90.1, 'energia_kcal': 31, 'proteina_g': 1.9, 
               'lipidios_g': 0.1, 'carboidratos_g': 7.2, 'fibra_g': 2.0, 'calcio_mg': 44, 
               'vitamina_c_mg': 18},
   'Tangerina': {'id': 251, 'umidade': 89.2, 'energia_kcal': 38, 'proteina_g': 0.8, 
                 'lipidios_g': 0.1, 'carboidratos_g': 9.6, 'fibra_g': 0.9, 'calcio_mg': 13, 
                 'vitamina_c_mg': 8},
   'Tomate': {'id': 157, 'umidade': 95.1, 'energia_kcal': 15, 'proteina_g': 1.1, 
              'lipidios_g': 0.2, 'carboidratos_g': 3.1, 'fibra_g': 1.2, 'calcio_mg': 7, 
              'vitamina_c_mg': 11},
   'Banana': {'id': 182, 'umidade': 71.9, 'energia_kcal': 98, 'proteina_g': 1.3, 
              'lipidios_g': 0.1, 'carboidratos_g': 26.0, 'fibra_g': 2.0, 'calcio_mg': 8, 
              'vitamina_c_mg': 26},
   # Produtos sem dados TACO específicos - usando aproximações
   'Berinjela': {'id': 9006, 'umidade': 92.5, 'energia_kcal': 24, 'proteina_g': 1.0, 
                 'lipidios_g': 0.2, 'carboidratos_g': 5.7, 'fibra_g': 3.4, 'calcio_mg': 9, 
                 'vitamina_c_mg': 2},
   'Quiabo': {'id': 9007, 'umidade': 90.0, 'energia_kcal': 33, 'proteina_g': 2.0, 
              'lipidios_g': 0.1, 'carboidratos_g': 7.0, 'fibra_g': 3.2, 'calcio_mg': 81, 
              'vitamina_c_mg': 21},
   'Jiló': {'id': 9008, 'umidade': 92.0, 'energia_kcal': 27, 'proteina_g': 1.4, 
            'lipidios_g': 0.2, 'carboidratos_g': 6.2, 'fibra_g': 4.8, 'calcio_mg': 20, 
            'vitamina_c_mg': 6},
   'Gengibre': {'id': 9009, 'umidade': 78.9, 'energia_kcal': 80, 'proteina_g': 1.8, 
                'lipidios_g': 0.8, 'carboidratos_g': 17.8, 'fibra_g': 2.0, 'calcio_mg': 16, 
                'vitamina_c_mg': 5},
   'Uva': {'id': 9010, 'umidade': 80.5, 'energia_kcal': 69, 'proteina_g': 0.7, 
           'lipidios_g': 0.2, 'carboidratos_g': 17.1, 'fibra_g': 0.9, 'calcio_mg': 10, 
           'vitamina_c_mg': 4},
   'Lichia': {'id': 9011, 'umidade': 81.8, 'energia_kcal': 66, 'proteina_g': 0.8, 
              'lipidios_g': 0.4, 'carboidratos_g': 16.5, 'fibra_g': 1.3, 'calcio_mg': 5, 
              'vitamina_c_mg': 72},
   'Pitaia': {'id': 9012, 'umidade': 87.0, 'energia_kcal': 50, 'proteina_g': 1.1, 
              'lipidios_g': 0.4, 'carboidratos_g': 11.0, 'fibra_g': 3.0, 'calcio_mg': 9, 
              'vitamina_c_mg': 21},
   'Mamão': {'id': 9013, 'umidade': 88.6, 'energia_kcal': 43, 'proteina_g': 0.5, 
             'lipidios_g': 0.3, 'carboidratos_g': 10.8, 'fibra_g': 1.8, 'calcio_mg': 20, 
             'vitamina_c_mg': 62},
   'Coco': {'id': 9014, 'umidade': 47.0, 'energia_kcal': 354, 'proteina_g': 3.3, 
            'lipidios_g': 33.5, 'carboidratos_g': 15.2, 'fibra_g': 9.0, 'calcio_mg': 14, 
            'vitamina_c_mg': 3},
   'Graviola': {'id': 9015, 'umidade': 81.2, 'energia_kcal': 66, 'proteina_g': 1.0, 
                'lipidios_g': 0.3, 'carboidratos_g': 16.8, 'fibra_g': 3.3, 'calcio_mg': 14, 
                'vitamina_c_mg': 21},
   'Cajamanga': {'id': 9016, 'umidade': 91.0, 'energia_kcal': 31, 'proteina_g': 0.6, 
                 'lipidios_g': 0.1, 'carboidratos_g': 7.6, 'fibra_g': 0.9, 'calcio_mg': 3, 
                 'vitamina_c_mg': 34},
   'Atemóia': {'id': 9017, 'umidade': 72.9, 'energia_kcal': 94, 'proteina_g': 1.7, 
               'lipidios_g': 0.3, 'carboidratos_g': 23.6, 'fibra_g': 2.4, 'calcio_mg': 24, 
               'vitamina_c_mg': 36},
   'Mandioca': {'id': 9018, 'umidade': 61.8, 'energia_kcal': 151, 'proteina_g': 1.0, 
                'lipidios_g': 0.3, 'carboidratos_g': 36.2, 'fibra_g': 1.9, 'calcio_mg': 19, 
                'vitamina_c_mg': 17}
}

# Criar DataFrame de nutrientes
df_nutrientes = pd.DataFrame.from_dict(dados_taco, orient='index')

# Adicionar categorias aos alimentos
categorias_alimentos = {
   'folhosas': ['Alface', 'Couve', 'Agrião'],
   'frutos_hortalicas': ['Tomate', 'Pimentão', 'Berinjela', 'Jiló', 'Quiabo', 'Chuchu', 'Pepino'],
   'raizes_tuberculos': ['Batata', 'Mandioca', 'Beterraba', 'Cenoura'],
   'bulbos': ['Cebola'],
   'cruciferas': ['Brócolis', 'Repolho'],
   'frutas_doces': ['Banana', 'Manga', 'Mamão', 'Uva', 'Lichia', 'Atemóia'],
   'frutas_acidas': ['Limão', 'Tangerina', 'Maracujá', 'Cajamanga'],
   'frutas_berries': ['Morango'],
   'frutas_gordurosas': ['Abacate', 'Coco'],
   'frutas_tropicais': ['Goiaba', 'Graviola', 'Pitaia'],
   'temperos': ['Gengibre'],
   'leguminosas': ['Abóbora']
}

# Inverter o dicionário para facilitar a atribuição
categoria_por_alimento = {}
for categoria, alimentos in categorias_alimentos.items():
   for alimento in alimentos:
       categoria_por_alimento[alimento] = categoria

df_nutrientes['categoria'] = df_nutrientes.index.map(categoria_por_alimento)

# Calcular scores nutricionais normalizados
df_nutrientes['score_vitamina_c'] = df_nutrientes['vitamina_c_mg'] / df_nutrientes['vitamina_c_mg'].max()
df_nutrientes['score_fibras'] = df_nutrientes['fibra_g'] / df_nutrientes['fibra_g'].max()
df_nutrientes['score_baixa_caloria'] = 1 - (df_nutrientes['energia_kcal'] / df_nutrientes['energia_kcal'].max())
df_nutrientes['score_proteina'] = df_nutrientes['proteina_g'] / df_nutrientes['proteina_g'].max()

print("📊 Dados nutricionais carregados com sucesso!")
print(f"Total de alimentos com dados nutricionais: {len(df_nutrientes)}")
print("\n🥗 Categorias de alimentos:")
for categoria, count in df_nutrientes['categoria'].value_counts().items():
   print(f"  - {categoria}: {count} alimentos")
print("\n💊 Top 5 alimentos por vitamina C:")
print(df_nutrientes.nlargest(5, 'vitamina_c_mg')[['vitamina_c_mg', 'categoria']])

In [ ]:
"""
Dados reais de produção por região do DF
Fonte: Informações Agropecuárias do Distrito Federal - 2024 (EMATER-DF)
"""

# Dados de produção por escritório/região
producao_emater = {
    'ALEXANDRE GUSMÃO': {
        'Abacate': {'area_ha': 23.08, 'producao_t': 18.0, 'participacao_df': 23.08},
        'Alface': {'area_ha': 24.25, 'producao_t': 9.51, 'participacao_df': 24.25},
        'Banana': {'area_ha': 9.5, 'producao_t': 6.83, 'participacao_df': 9.5},
        'Chuchu': {'area_ha': 32.99, 'producao_t': 14.52, 'participacao_df': 32.99},
        'Couve': {'area_ha': 35.66, 'producao_t': 9.39, 'participacao_df': 35.66},
        'Goiaba': {'area_ha': 42.2, 'producao_t': 38.56, 'participacao_df': 42.2},
        'Limão': {'area_ha': 13.15, 'producao_t': 12.04, 'participacao_df': 13.15},
        'Mandioca': {'area_ha': 18.29, 'producao_t': 6.17, 'participacao_df': 18.29},
        'Manga': {'area_ha': 13.67, 'producao_t': 1.66, 'participacao_df': 13.67},
        'Maracujá': {'area_ha': 12.19, 'producao_t': 10.26, 'participacao_df': 12.19},
        'Pimentão': {'area_ha': 14.73, 'producao_t': 4.13, 'participacao_df': 14.73},
        'Repolho': {'area_ha': 24.68, 'producao_t': 8.87, 'participacao_df': 24.68},
        'Tangerina': {'area_ha': 21.44, 'producao_t': 6.96, 'participacao_df': 21.44},
        'Tomate': {'area_ha': 14.62, 'producao_t': 11.67, 'participacao_df': 14.62},
        'Uva': {'area_ha': 4.16, 'producao_t': 0.77, 'participacao_df': 4.16}
    },
    'BRAZLÂNDIA': {
        'Abacate': {'area_ha': 19.45, 'producao_t': 18.6, 'participacao_df': 19.45},
        'Alface': {'area_ha': 8.87, 'producao_t': 5.95, 'participacao_df': 8.87},
        'Banana': {'area_ha': 4.35, 'producao_t': 3.61, 'participacao_df': 4.35},
        'Chuchu': {'area_ha': 27.72, 'producao_t': 13.09, 'participacao_df': 27.72},
        'Couve': {'area_ha': 9.43, 'producao_t': 3.24, 'participacao_df': 9.43},
        'Goiaba': {'area_ha': 53.62, 'producao_t': 57.58, 'participacao_df': 53.62},
        'Limão': {'area_ha': 9.3, 'producao_t': 9.48, 'participacao_df': 9.3},
        'Mandioca': {'area_ha': 6.88, 'producao_t': 4.08, 'participacao_df': 6.88},
        'Manga': {'area_ha': 4.4, 'producao_t': 0.6, 'participacao_df': 4.4},
        'Maracujá': {'area_ha': 3.94, 'producao_t': 1.38, 'participacao_df': 3.94},
        'Morango': {'area_ha': 76.51, 'producao_t': 17.36, 'participacao_df': 76.51},
        'Pimentão': {'area_ha': 15.89, 'producao_t': 6.18, 'participacao_df': 15.89},
        'Repolho': {'area_ha': 23.53, 'producao_t': 10.6, 'participacao_df': 23.53},
        'Tangerina': {'area_ha': 11.42, 'producao_t': 5.63, 'participacao_df': 11.42},
        'Tomate': {'area_ha': 8.66, 'producao_t': 10.33, 'participacao_df': 8.66},
        'Uva': {'area_ha': 3.29, 'producao_t': 0.24, 'participacao_df': 3.29}
    },
    'CEILÂNDIA': {
        'Abacate': {'area_ha': 3.61, 'producao_t': 9.77, 'participacao_df': 3.61},
        'Alface': {'area_ha': 24.1, 'producao_t': 12.04, 'participacao_df': 24.1},
        'Banana': {'area_ha': 12.33, 'producao_t': 29.03, 'participacao_df': 12.33},
        'Chuchu': {'area_ha': 29.16, 'producao_t': 17.64, 'participacao_df': 29.16},
        'Couve': {'area_ha': 22.59, 'producao_t': 7.41, 'participacao_df': 22.59},
        'Goiaba': {'area_ha': 0.37, 'producao_t': 1.44, 'participacao_df': 0.37},
        'Limão': {'area_ha': 10.43, 'producao_t': 23.69, 'participacao_df': 10.43},
        'Mandioca': {'area_ha': 16.23, 'producao_t': 9.48, 'participacao_df': 16.23},
        'Manga': {'area_ha': 5.92, 'producao_t': 3.63, 'participacao_df': 5.92},
        'Maracujá': {'area_ha': 7.96, 'producao_t': 20.18, 'participacao_df': 7.96},
        'Repolho': {'area_ha': 17.83, 'producao_t': 8.75, 'participacao_df': 17.83},
        'Tangerina': {'area_ha': 6.03, 'producao_t': 9.55, 'participacao_df': 6.03},
        'Tomate': {'area_ha': 3.22, 'producao_t': 3.47, 'participacao_df': 3.22}
    },
    'GAMA': {
        'Abacate': {'area_ha': 6.98, 'producao_t': 20.49, 'participacao_df': 6.98},
        'Alface': {'area_ha': 19.58, 'producao_t': 32.92, 'participacao_df': 19.58},
        'Banana': {'area_ha': 5.59, 'producao_t': 15.58, 'participacao_df': 5.59},
        'Couve': {'area_ha': 9.74, 'producao_t': 8.83, 'participacao_df': 9.74},
        'Lichia': {'area_ha': 5.86, 'producao_t': 1.21, 'participacao_df': 5.86},
        'Limão': {'area_ha': 12.17, 'producao_t': 27.56, 'participacao_df': 12.17},
        'Mandioca': {'area_ha': 9.66, 'producao_t': 16.37, 'participacao_df': 9.66},
        'Manga': {'area_ha': 13.1, 'producao_t': 6.94, 'participacao_df': 13.1},
        'Maracujá': {'area_ha': 3.06, 'producao_t': 7.6, 'participacao_df': 3.06},
        'Pimentão': {'area_ha': 2.24, 'producao_t': 3.26, 'participacao_df': 2.24},
        'Pitaia': {'area_ha': 16.31, 'producao_t': 1.71, 'participacao_df': 16.31},
        'Repolho': {'area_ha': 1.85, 'producao_t': 2.31, 'participacao_df': 1.85},
        'Tangerina': {'area_ha': 9.07, 'producao_t': 10.7, 'participacao_df': 9.07}
    },
    'PLANALTINA': {
        'Abacate': {'area_ha': 4.26, 'producao_t': 8.23, 'participacao_df': 4.26},
        'Alface': {'area_ha': 1.13, 'producao_t': 3.94, 'participacao_df': 1.13},
        'Banana': {'area_ha': 7.35, 'producao_t': 10.82, 'participacao_df': 7.35},
        'Chuchu': {'area_ha': 1.94, 'producao_t': 4.52, 'participacao_df': 1.94},
        'Couve': {'area_ha': 0.96, 'producao_t': 1.69, 'participacao_df': 0.96},
        'Goiaba': {'area_ha': 0.8, 'producao_t': 2.57, 'participacao_df': 0.8},
        'Lichia': {'area_ha': 61.0, 'producao_t': 7.17, 'participacao_df': 61.0},
        'Limão': {'area_ha': 15.68, 'producao_t': 27.24, 'participacao_df': 15.68},
        'Mandioca': {'area_ha': 7.19, 'producao_t': 30.89, 'participacao_df': 7.19},
        'Manga': {'area_ha': 13.01, 'producao_t': 3.23, 'participacao_df': 13.01},
        'Maracujá': {'area_ha': 7.53, 'producao_t': 11.02, 'participacao_df': 7.53},
        'Pimentão': {'area_ha': 2.1, 'producao_t': 2.39, 'participacao_df': 2.1},
        'Repolho': {'area_ha': 1.68, 'producao_t': 3.74, 'participacao_df': 1.68},
        'Tangerina': {'area_ha': 10.55, 'producao_t': 8.73, 'participacao_df': 10.55},
        'Tomate': {'area_ha': 3.49, 'producao_t': 16.83, 'participacao_df': 3.49},
        'Uva': {'area_ha': 15.24, 'producao_t': 11.13, 'participacao_df': 15.24}
    },
    'SOBRADINHO': {
        'Abacate': {'area_ha': 5.78, 'producao_t': 11.67, 'participacao_df': 5.78},
        'Alface': {'area_ha': 11.79, 'producao_t': 12.59, 'participacao_df': 11.79},
        'Banana': {'area_ha': 27.36, 'producao_t': 43.04, 'participacao_df': 27.36},
        'Berinjela': {'area_ha': 14.33, 'producao_t': 4.35, 'participacao_df': 14.33},
        'Chuchu': {'area_ha': 2.32, 'producao_t': 3.22, 'participacao_df': 2.32},
        'Couve': {'area_ha': 9.15, 'producao_t': 5.88, 'participacao_df': 9.15},
        'Limão': {'area_ha': 7.6, 'producao_t': 11.29, 'participacao_df': 7.6},
        'Mandioca': {'area_ha': 15.39, 'producao_t': 14.95, 'participacao_df': 15.39},
        'Manga': {'area_ha': 25.42, 'producao_t': 10.15, 'participacao_df': 25.42},
        'Maracujá': {'area_ha': 8.8, 'producao_t': 3.45, 'participacao_df': 8.8},
        'Pimentão': {'area_ha': 5.13, 'producao_t': 6.43, 'participacao_df': 5.13},
        'Repolho': {'area_ha': 11.8, 'producao_t': 12.24, 'participacao_df': 11.8},
        'Tangerina': {'area_ha': 7.6, 'producao_t': 6.44, 'participacao_df': 7.6},
        'Tomate': {'area_ha': 4.5, 'producao_t': 8.75, 'participacao_df': 4.5},
        'Uva': {'area_ha': 22.17, 'producao_t': 5.35, 'participacao_df': 22.17}
    },
    'SÃO SEBASTIÃO': {
        'Abacate': {'area_ha': 0.1, 'producao_t': 1.86, 'participacao_df': 0.1},
        'Abóbora': {'area_ha': 4.14, 'producao_t': 0.19, 'participacao_df': 4.14},
        'Agrião': {'area_ha': 0.94, 'producao_t': 0.1, 'participacao_df': 0.94},
        'Alface': {'area_ha': 0.46, 'producao_t': 0.34, 'participacao_df': 0.46},
        'Banana': {'area_ha': 1.5, 'producao_t': 28.78, 'participacao_df': 1.5},
        'Cebola': {'area_ha': 66.0, 'producao_t': 95.49, 'participacao_df': 66.0},
        'Couve': {'area_ha': 0.59, 'producao_t': 0.22, 'participacao_df': 0.59},
        'Limão': {'area_ha': 1.05, 'producao_t': 11.77, 'participacao_df': 1.05},
        'Mandioca': {'area_ha': 2.6, 'producao_t': 1.98, 'participacao_df': 2.6},
        'Manga': {'area_ha': 5.63, 'producao_t': 20.2, 'participacao_df': 5.63},
        'Maracujá': {'area_ha': 8.45, 'producao_t': 28.31, 'participacao_df': 8.45},
        'Morango': {'area_ha': 2.41, 'producao_t': 0.46, 'participacao_df': 2.41},
        'Pitaia': {'area_ha': 0.37, 'producao_t': 0.31, 'participacao_df': 0.37},
        'Quiabo': {'area_ha': 1.16, 'producao_t': 0.09, 'participacao_df': 1.16},
        'Repolho': {'area_ha': 0.1, 'producao_t': 0.09, 'participacao_df': 0.1},
        'Tangerina': {'area_ha': 0.27, 'producao_t': 1.36, 'participacao_df': 0.27},
        'Tomate': {'area_ha': 0.17, 'producao_t': 0.18, 'participacao_df': 0.17}
    }
}

# Converter para DataFrame
lista_producao = []
for regiao, produtos in producao_emater.items():
    for produto, dados in produtos.items():
        lista_producao.append({
            'regiao': regiao,
            'produto': produto,
            'area_ha': dados['area_ha'],
            'producao_t': dados['producao_t'],
            'participacao_df': dados['participacao_df']
        })